In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load the datasets
districts_df = pd.read_csv('maharashtra_districts_nutrients.csv')
crops_df = pd.read_csv('maharashtra_crops_nutrients.csv')
districts_df.head()
# crops_df.head()

,District,Nitrogen,Phosphorus,Potassium
0,Mumbai City,150,80,120
1,Pune,180,60,100
2,Nagpur,160,70,110
3,Nashik,170,50,90
4,Solapur,140,90,130


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import os

# Load the datasets
districts_df = pd.read_csv('maharashtra_districts_nutrients.csv')
crops_df = pd.read_csv('maharashtra_crops_nutrients.csv')

# Function to prepare data
def prepare_data(districts_df, crops_df):
    merged_df = pd.merge(districts_df, crops_df, how='cross', suffixes=('_district', '_crop'))

    # Calculate nutrient sufficiency
    merged_df['Nitrogen_Sufficient'] = merged_df['Nitrogen_district'] >= merged_df['Nitrogen_crop']
    merged_df['Phosphorus_Sufficient'] = merged_df['Phosphorus_district'] >= merged_df['Phosphorus_crop']
    merged_df['Potassium_Sufficient'] = merged_df['Potassium_district'] >= merged_df['Potassium_crop']

    # Determine if a crop is suitable based on nutrient sufficiency
    merged_df['Suitable'] = merged_df[['Nitrogen_Sufficient', 'Phosphorus_Sufficient', 'Potassium_Sufficient']].all(axis=1)

    return merged_df

# Function to train the model
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Save the model and scaler
    joblib.dump(model, 'crop_suitability_model.pkl')

    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_test, y_pred))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Suitable', 'Suitable'], yticklabels=['Not Suitable', 'Suitable'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    return model

# Function to load the model
def load_model():
    model = joblib.load('crop_suitability_model.pkl')
    return model

# Function to suggest fertilizers
fertilizers = {
    'Nitrogen': 'Urea',
    'Phosphorus': 'Single Super Phosphate (SSP)',
    'Potassium': 'Muriate of Potash (MOP)'
}

def suggest_fertilizers(nutrient_gap):
    suggestions = {}
    for nutrient, gap in nutrient_gap.items():
        if gap > 0:
            suggestions[nutrient] = {
                'Fertilizer': fertilizers[nutrient],
                'Amount': gap
            }
    return suggestions

# Function to predict crop suitability and suggest fertilizers if needed
def predict_crop_suitability(model, district, crop, districts_df, crops_df, user_nutrients=None):
    if user_nutrients:
        nitrogen, phosphorus, potassium = user_nutrients
    else:
        district_data = districts_df[districts_df['District'].str.lower() == district.lower()].iloc[0]
        nitrogen = district_data['Nitrogen']
        phosphorus = district_data['Phosphorus']
        potassium = district_data['Potassium']

    crop_data = crops_df[crops_df['Crop'].str.lower() == crop.lower()].iloc[0]

    input_data = pd.DataFrame([[
        nitrogen,
        phosphorus,
        potassium
    ]], columns=['Nitrogen_district', 'Phosphorus_district', 'Potassium_district'])

    is_suitable = model.predict(input_data)[0]

    nutrient_gap = {
        'Nitrogen': crop_data['Nitrogen'] - nitrogen,
        'Phosphorus': crop_data['Phosphorus'] - phosphorus,
        'Potassium': crop_data['Potassium'] - potassium
    }

    threshold = 20
    if is_suitable or not any(gap > threshold for gap in nutrient_gap.values()):
        suggestions = suggest_fertilizers(nutrient_gap)
        return f"The crop '{crop}' is suitable for the district '{district}' with the suggested fertilizers: {suggestions}."
    else:
        return f"The crop '{crop}' is not suitable for the district '{district}' due to high nutrient deficiencies."

# Function to get crops suitable and unsuitable for a district
def get_crops_for_district(model, district, districts_df, crops_df, user_nutrients=None):
    if user_nutrients:
        nitrogen, phosphorus, potassium = user_nutrients
    else:
        district_data = districts_df[districts_df['District'].str.lower() == district.lower()]
        if district_data.empty:
            return f"The district '{district}' was not found in the dataset."
        district_data = district_data.iloc[0]
        nitrogen = district_data['Nitrogen']
        phosphorus = district_data['Phosphorus']
        potassium = district_data['Potassium']

    suitable_crops = []
    unsuitable_crops = []
    unsuitable_crops_info = []

    for _, crop_data in crops_df.iterrows():
        crop = crop_data['Crop']
        input_data = pd.DataFrame([[
            nitrogen,
            phosphorus,
            potassium
        ]], columns=['Nitrogen_district', 'Phosphorus_district', 'Potassium_district'])

        is_suitable = model.predict(input_data)[0]

        nutrient_gap = {
            'Nitrogen': crop_data['Nitrogen'] - nitrogen,
            'Phosphorus': crop_data['Phosphorus'] - phosphorus,
            'Potassium': crop_data['Potassium'] - potassium
        }

        threshold = 20
        if is_suitable or not any(gap > threshold for gap in nutrient_gap.values()):
            suitable_crops.append(crop)
        else:
            if any(gap > threshold for gap in nutrient_gap.values()):
                unsuitable_crops_info.append((crop, "High nutrient deficiencies"))
            else:
                suggestions = suggest_fertilizers(nutrient_gap)
                unsuitable_crops_info.append((crop, suggestions))

            unsuitable_crops.append(crop)

    return suitable_crops, unsuitable_crops, unsuitable_crops_info

# Function to get user input for nutrient levels
def get_user_nutrients():
    try:
        nitrogen = float(input("Enter the nitrogen level: "))
        phosphorus = float(input("Enter the phosphorus level: "))
        potassium = float(input("Enter the potassium level: "))
        return (nitrogen, phosphorus, potassium)
    except ValueError:
        print("Invalid input. Please enter numeric values.")
        return get_user_nutrients()

# Main function to run the entire program
def main():
    if os.path.exists('crop_suitability_model.pkl'):
        model = load_model()
        print("Model loaded from file.")
    else:
        merged_df = prepare_data(districts_df, crops_df)
        X = merged_df[['Nitrogen_district', 'Phosphorus_district', 'Potassium_district']]
        y = merged_df['Suitable']
        model = train_model(X, y)

    district = input("Enter the district name: ").strip().lower()
    crop = input("Enter the crop name: ").strip().lower()

    user_input = input("Do you want to provide custom nutrient levels? (yes/no): ").strip().lower()
    if user_input == 'yes':
        user_nutrients = get_user_nutrients()
    else:
        user_nutrients = None

    result = predict_crop_suitability(model, district, crop, districts_df, crops_df, user_nutrients)
    print(result)

    suitable_crops, unsuitable_crops, unsuitable_crops_info = get_crops_for_district(model, district, districts_df, crops_df, user_nutrients)
    print(f"Suitable crops for district '{district}': {suitable_crops}")
    print(f"Unsuitable crops for district '{district}': {unsuitable_crops}")
    print("Details of unsuitable crops:")
    for crop_info in unsuitable_crops_info:
        print(crop_info)

if __name__ == "__main__":
    main()

Model loaded from file.


Enter the district name:  Nashik
Enter the crop name:  jowar
Do you want to provide custom nutrient levels? (yes/no):  no


The crop 'jowar' is suitable for the district 'nashik' with the suggested fertilizers: {'Nitrogen': {'Fertilizer': 'Urea', 'Amount': 10}}.
Suitable crops for district 'nashik': ['Jowar', 'Rice', 'Grapes', 'Soybean', 'Green Gram', 'Banana', 'Orange', 'Apple', 'Guava', 'Chikoo', 'Pineapple', 'Watermelon', 'Muskmelon', 'Onion', 'Potato', 'Tomato', 'Brinjal', 'Cabbage', 'Methi', 'Fenugreek', 'Coriander', 'Mint']
Unsuitable crops for district 'nashik': ['Wheat', 'Bajra', 'Cotton', 'Sugarcane', 'Turmeric', 'Mango', 'Pomegranate', 'Cauliflower', 'Broccoli', 'Spinach']
Details of unsuitable crops:
('Wheat', 'High nutrient deficiencies')
('Bajra', 'High nutrient deficiencies')
('Cotton', 'High nutrient deficiencies')
('Sugarcane', 'High nutrient deficiencies')
('Turmeric', 'High nutrient deficiencies')
('Mango', 'High nutrient deficiencies')
('Pomegranate', 'High nutrient deficiencies')
('Cauliflower', 'High nutrient deficiencies')
('Broccoli', 'High nutrient deficiencies')
('Spinach', 'High nu